In [7]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir, 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from modules import dataloading as dl
from modules import cleaning as c
import pandas as pd
targetdir = '../../src/data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# import data'
apr_2020_df = dl.CPS_raw(targetdir, ['apr2020'], None)
jul_2020_df = dl.CPS_raw(targetdir, ['jul2020'], None)
# jan_feb_2020_df = dl.CPS_raw(targetdir, ['jan2020', 'feb2020'], None)


In [41]:
apr_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129382 entries, 0 to 129381
Columns: 391 entries, HRHHID to IND_ID_FINAL
dtypes: int64(387), object(4)
memory usage: 386.0+ MB


# We decided on which columns to keep and which to discard

The target variable will be change in employment status from having a job to not. We could also make a conditional where if the they WHERE full time (40 hours+) but now are not, that could also represent a change in the employment status.


But if our question is: How many people lost full time work during the first wave of the pandemic, The data set becomes much simpler.

In [68]:
to_keep = [
    'PREMPNOT', # EMPLOY TARGET VARIABLE
    'PEMLR', # TARGET VARIABLE 1-6 employment status
    'IND_ID_FINAL', # the unique identifier of the person
    'PRPERTYP', # Type of person recode 
    'HEHOUSUT', # type of housing unit to dummy
    'HETENURE', # rent or morgage status
    'HWHHWGT', # Household weight
    'GESTFIPS', # state codes
    "GTMETSTA", # Metropolitan or not
    'HEFAMINC', # total family income
    "HRNUMHOU", # total number of people living in the house hold
    'HRHTYPE', # household type eg civilian or married etc
    'PERRP', # relationship to reference, -1, 40 and 41
    'PRTAGE', # person's age
    'PEMARITL', # marital status
    'PESPOUSE', # line number of spouse (-1) == no spouse
    'PESEX', # gender 1 == male, 2 == female
    'PEEDUCA', # level of education see dict for coding
    'PTDTRACE', # race composition of the house. See data dict
    "PEHSPNON", # hispanic or not hispanic
    'PUCHINHH', # change in household composition. See data dict
    'PENATVTY', # country of birth ie US born or not
    "PEMNTVTY", # mothers country of birth, 
    "PEFNTVTY", # fathers country of birth ie are they a first generation imiigrant?
    "PRCITSHP", # citezen status
    "PRINUSYR", # Year since immigration -1== us born, else coded by decade
    "PUABSOT", # Did you have a job last week TARGET POTENTIAL
    "PULAY", # Where you on layoff from your job last week?
    "PEABSRSN", # reason for not being at work the day before.
    "PRHRUSL", # Hours at work, dummy into full time or not full time
    "HUBUS", # Does anyone have a business or a farm?
    "PEMJOT", # Do you have more than 1 job?
    "PEMJNUM", # how many jobs do you have?
    "PEHRFTPT", # Do you normally spend more than 35 hours a week at your main job?
    "PEHRRSN2", # what is the main reason you do not want to work 35 hours. Speaks to motivation of keeping job.
    "PEHRACTT", # sum of hours worked between all jobs
    "PRCIVLF", # in the civiliarn labour force = 1 
    "PRAGNA", # Agricultural industry yes or no
    "PRMJIND1", # industry recode
    "PEERNUOT", # Do you usually recieve tips at your work
    "PEERNHRY", # hourly vs non-hourly status
    "PRERNWA", # weekly earnings
    "PEERNLAB", # part of a labour union
    "PRNLFSCH", # In school or not
    "PRNMCHLD", # number of children less than 18 years old (-1 not a parent, 0-99 number of children)
    "PECYC", # How much college credit in years has the reference person recieved?
    "PEIO1ICD", # Inustry code for primary jobs lin1
    "PEIO1OCD", # Occupation code for primary jobs lin1
    "PECERT1", # Do you have a professional certification issued at state or federal level.
]

In [69]:
len(to_keep)

49

In [ ]:
#"PRMJIND1" & PRMJIND2,  industry codes Primary business industry job 1

"""
1 Agriculture, forestry, fishing, and hunting\
2 Mining
3 Construction
4 Manufacturing
5 Wholesale and retail trade
6 Transportation and utilities      
7 Information
8 Financial activities
9 Professional and business services
10 Educational and health services
11 Leisure and hospitality
12 Other services
13 Public administration
14 Armed Forces"""

In [67]:
"""U.S. Census Bureau					
2020 Current Population Survey Public Use Industry Code List					
					
	*  0010 - 0560  Agriculture, Forestry, Fishing and Hunting, and Mining				
	*  0770 - 1060  Construction				
	*  1070 - 4060  Manufacturing				
	*  4070 - 4660  Wholesale Trade				
	*  4670 - 6060  Retail Trade				
	*  6070 - 6460, 0570 - 0760  Transportation and Warehousing, and Utilities				
	*  6470 - 6860  Information				
	*  6870 - 7260  Finance and Insurance,  and Real Estate and Rental and Leasing				
	*  7270 - 7790  Professional, Scientific, and Management, and  Administrative and Waste Management Services				
	*  7860 - 8490  Educational Services, and Health Care and Social Assistance				
	*  8560 - 8690  Arts, Entertainment, and Recreation, and  Accommodation and Food Services				
	*  8770 - 9290  Other Services (except Public Administration)				
	*  9370 - 9590  Public Administration				
	*  9890         Military	"""

'U.S. Census Bureau\t\t\t\t\t\n2020 Current Population Survey Public Use Industry Code List\t\t\t\t\t\n\t\t\t\t\t\n\t*  0010 - 0560  Agriculture, Forestry, Fishing and Hunting, and Mining\t\t\t\t\n\t*  0770 - 1060  Construction\t\t\t\t\n\t*  1070 - 4060  Manufacturing\t\t\t\t\n\t*  4070 - 4660  Wholesale Trade\t\t\t\t\n\t*  4670 - 6060  Retail Trade\t\t\t\t\n\t*  6070 - 6460, 0570 - 0760  Transportation and Warehousing, and Utilities\t\t\t\t\n\t*  6470 - 6860  Information\t\t\t\t\n\t*  6870 - 7260  Finance and Insurance,  and Real Estate and Rental and Leasing\t\t\t\t\n\t*  7270 - 7790  Professional, Scientific, and Management, and  Administrative and Waste Management Services\t\t\t\t\n\t*  7860 - 8490  Educational Services, and Health Care and Social Assistance\t\t\t\t\n\t*  8560 - 8690  Arts, Entertainment, and Recreation, and  Accommodation and Food Services\t\t\t\t\n\t*  8770 - 9290  Other Services (except Public Administration)\t\t\t\t\n\t*  9370 - 9590  Public Administration\t\t\

In [20]:
apr_cleaned = c.clean_CPS_df(apr_2020_df)
jul_cleaned = c.clean_CPS_df(jul_2020_df)

In [60]:
apr_cleaned.PEMLR.value_counts()

-1    45867
 1    38284
 5    18071
 7    13070
 3     5278
 6     4109
 2     3426
 4     1277
Name: PEMLR, dtype: int64

In [26]:
jul_cleaned.PREMPNOT.value_counts()#.sum()

-1    45293
 1    42419
 4    31861
 2     4360
 3      169
Name: PREMPNOT, dtype: int64

In [25]:
apr_cleaned.PREMPNOT.value_counts()#.sum()

-1    45867
 1    41710
 4    35086
 2     6555
 3      164
Name: PREMPNOT, dtype: int64

In [30]:
jul_cleaned.PRUNTYPE.value_counts()

-1    119742
 1      2474
 2       715
 5       604
 6       204
 3       182
 4       181
Name: PRUNTYPE, dtype: int64

In [34]:
2474/jul_cleaned.PRUNTYPE.value_counts().sum()

0.019935214581553885

In [32]:
apr_cleaned.PRUNTYPE.value_counts()

-1    122827
 1      5278
 2       536
 5       361
 4       153
 3       153
 6        74
Name: PRUNTYPE, dtype: int64

In [36]:
5278/apr_cleaned.PRUNTYPE.value_counts().sum()

0.04079392805799879

In [48]:
df2 = apr_cleaned.merge(jul_cleaned, on='IND_ID_FINAL')

In [51]:
df2

,HRHHID_x,HRMONTH_x,HRYEAR4_x,HURESPLI_x,HUFINAL_x,HETENURE_x,HEHOUSUT_x,HETELHHD_x,HETELAVL_x,HEPHONEO_x,...,PTNMEMP2_y,PECERT1_y,PECERT2_y,PECERT3_y,PXCERT1_y,PXCERT2_y,PXCERT3_y,HH_ID_y,IND_ID_y,IND_ID_SEX_AGE_y
0,110500762096,4,2020,-1,219,-1,1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,11050076209611011,11050076209611011-1,11050076209611011-1-1-1
1,110761517565,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,11076151756511111,1107615175651111141,1107615175651111141258
2,1051765008631,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,105176500863110111,10517650086311011141,10517650086311011141247
3,1710617055156,4,2020,-1,226,-1,1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,171061705515611111,171061705515611111-1,171061705515611111-1-1-1
4,6006205110907,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,600620511090711011,60062051109071101140,60062051109071101140231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18650,810003625589536,4,2020,1,201,-1,1,1,-1,0,...,-1,2,-1,-1,20,0,0,81000362558953610011,8100036255895361001151,8100036255895361001151232
18651,810103625129536,4,2020,-1,219,-1,1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,81010362512953611011,81010362512953611011-1,81010362512953611011-1-1-1
18652,902527115610997,4,2020,1,201,-1,1,1,-1,1,...,-1,1,1,2,20,20,0,90252711561099711011,9025271156109971101141,9025271156109971101141262
18653,906526256610777,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,90652625661077711011,9065262566107771101141,9065262566107771101141175


In [54]:
df2[(df2.PREMPNOT_x == 1) & (df2.PREMPNOT_y == 2) & (df2.PERRP_x.isin([40,41,-1]))]


,HRHHID_x,HRMONTH_x,HRYEAR4_x,HURESPLI_x,HUFINAL_x,HETENURE_x,HEHOUSUT_x,HETELHHD_x,HETELAVL_x,HEPHONEO_x,...,PTNMEMP2_y,PECERT1_y,PECERT2_y,PECERT3_y,PXCERT1_y,PXCERT2_y,PXCERT3_y,HH_ID_y,IND_ID_y,IND_ID_SEX_AGE_y
56,731618601050005,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,73161860105000510111,7316186010500051011140,7316186010500051011140128
487,610209360112252,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,61020936011225210011,6102093601122521001141,6102093601122521001141132
757,575014544001181,4,2020,1,201,-1,1,1,1,1,...,-1,2,-1,-1,20,0,0,57501454400118110011,5750145440011811001140,5750145440011811001140157
1128,17035346005591,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,1703534600559111011,170353460055911101140,170353460055911101140152
1326,1867029102890,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,186702910289010011,18670291028901001141,18670291028901001141151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17657,50411992306912,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,5041199230691210111,504119923069121011141,504119923069121011141126
18196,50510312874919,4,2020,1,201,-1,1,1,-1,1,...,-1,2,-1,-1,20,0,0,5051031287491911011,505103128749191101140,505103128749191101140253
18293,955214140504204,4,2020,1,201,-1,1,1,-1,1,...,-1,1,1,1,20,20,20,95521414050420410011,9552141405042041001141,9552141405042041001141156
18350,27056034504621,4,2020,2,201,-1,5,1,-1,1,...,-1,2,-1,-1,20,0,0,2705603450462110111,270560345046211011140,270560345046211011140223


In [57]:
df3 = df2[df2.PERRP_x.isin([40,41,-1])]

In [58]:
apr_cleaned.HWHHWGT

0         17968107
1         29235866
2         29235866
3         29235866
4         17922691
            ...   
129377          -1
129378     4955159
129379          -1
129380          -1
129381          -1
Name: HWHHWGT, Length: 129382, dtype: int64